In [4]:
import requests
from bs4 import BeautifulSoup

def crawl_vnexpress_articles(num_articles=50):
    url = "https://vnexpress.net"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    articles = []
    links = []

    # Lấy các thẻ chứa đường link bài viết chính
    for a in soup.select("a[href^='https://vnexpress.net']"):
        href = a.get("href")
        if href and href not in links and len(links) < num_articles:
            links.append(href)

    print(f"Đã thu thập {len(links)} link bài viết.")

    for link in links:
        try:
            r = requests.get(link, timeout=5)
            s = BeautifulSoup(r.text, "html.parser")

            # Lấy nội dung chính (thẻ article)
            body = s.find("article")
            if body:
                paragraphs = body.find_all("p")
                content = "\n".join([p.get_text() for p in paragraphs])
                if len(content) > 100:
                    articles.append(content)
        except:
            continue

    # Gộp toàn bộ nội dung và lưu
    full_text = "\n".join(articles)
    with open("raw_articles.txt", "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"Đã lưu {len(articles)} bài viết vào file raw_articles.txt")

# Gọi hàm crawl
crawl_vnexpress_articles()


Đã thu thập 50 link bài viết.
Đã lưu 50 bài viết vào file raw_articles.txt


In [5]:
import re
from underthesea import word_tokenize
import os

# 1. Đưa về chữ thường, loại bỏ dấu câu, số, ký tự đặc biệt, loại bỏ khoảng trắng dư thừa
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ'
                  r'ìíịỉĩòóọỏõôồốộổỗơờớợởỡ'
                  r'ùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# 2. Đọc dữ liệu thô và làm sạch
with open('raw_articles.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
cleaned_text = clean_text(raw_text)

# 3. Tách từ bằng underthesea
tokenized_text = word_tokenize(cleaned_text, format="text")
tokens = tokenized_text.replace("_", " ").split()

# 4. Lọc từ tiếng Việt
def load_all_vietnamese_words(folder_path):
    all_words = set()
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    word = line.strip().lower()
                    if word:
                        all_words.add(word)
    return all_words

folder_path = 'tu_dien_goc'  # Thư mục chứa các file từ điển
vietnamese_dict = load_all_vietnamese_words(folder_path)
vietnamese_only_tokens = [word for word in tokens if word in vietnamese_dict]

# 5. Loại bỏ từ trùng lặp và sắp xếp theo thứ tự abc
unique_sorted = sorted(set(vietnamese_only_tokens), key=lambda x: x.lower())

# 6. Lưu kết quả ra file
with open('vietnamese_only_tokens_sorted.txt', 'w', encoding='utf-8') as f:
    for word in unique_sorted:
        f.write(word + '\n')

print(f"Đã lọc và sắp xếp {len(unique_sorted)} từ tiếng Việt duy nhất.")

Đã lọc và sắp xếp 1646 từ tiếng Việt duy nhất.


In [ ]:
import random
import unicodedata

vowels = "aăâeêioôơuưy"
tone_marks = ["́", "̀", "̉", "̃", "̣"]

def remove_tone(char):
    decomposed = unicodedata.normalize('NFD', char)
    base = ''.join([c for c in decomposed if c not in tone_marks])
    return unicodedata.normalize('NFC', base)

def random_tone(char):
    if char not in vowels:
        return char
    base = unicodedata.normalize('NFD', char)[0]
    tone = random.choice(tone_marks)
    return unicodedata.normalize('NFC', base + tone)

def confused_initials(word):
    # Các nhóm phụ âm đầu dễ nhầm lẫn
    confusion_groups = [
        ["ch", "tr"],
        ["l", "n"],
        ["d", "r", "gi"],
        ["s", "x"],
        ["c", "k"],
        ["ngh", "ng","gh"],
    ]
    results = []
    for group in confusion_groups:
        for prefix in group:
            if word.startswith(prefix):
                for alt in group:
                    if alt != prefix:
                        results.append(alt + word[len(prefix):])
                return results  # chỉ đổi 1 nhóm đầu tiên tìm thấy
    return []

def generate_typos(word, num_typos=10):
    typo_list = []
    operations = [
        "repeat",      # Thừa chữ
        "delete",      # Thiếu chữ
        "remove_tone", # Thiếu dấu
        "wrong_tone",  # Sai dấu
        "transpose",   # Sai thứ tự chữ
        "confused_initial", # Nhầm phụ âm đầu
    ]
    for _ in range(num_typos):
        op = random.choice(operations)
        typo = None
        if op == "repeat" and len(word) > 0:
            pos = random.randint(0, len(word) - 1)
            typo = word[:pos] + word[pos] + word[pos:]
        elif op == "delete" and len(word) > 1:
            pos = random.randint(0, len(word) - 1)
            typo = word[:pos] + word[pos+1:]
        elif op == "remove_tone" and any(c in vowels for c in word):
            pos = random.choice([i for i, c in enumerate(word) if c in vowels])
            typo = word[:pos] + remove_tone(word[pos]) + word[pos+1:]
        elif op == "wrong_tone" and any(c in vowels for c in word):
            pos = random.choice([i for i, c in enumerate(word) if c in vowels])
            typo = word[:pos] + random_tone(word[pos]) + word[pos+1:]
        elif op == "transpose" and len(word) > 1:
            pos = random.randint(0, len(word) - 2)
            typo = word[:pos] + word[pos+1] + word[pos] + word[pos+2:]
        elif op == "confused_initial":
            confused = confused_initials(word)
            if confused:
                typo = random.choice(confused)
        if typo and typo != word and typo not in typo_list:
            typo_list.append(typo)
    return typo_list

In [ ]:
# Đọc danh sách từ
with open("vietnamese_only_tokens_sorted.txt", "r", encoding="utf-8") as f:
    words = [line.strip() for line in f if line.strip()]

# Sinh lỗi cho từng từ và lưu ra file
with open("typos_for_all_words.txt", "w", encoding="utf-8") as f:
    for word in words:
        typos = generate_typos(word, num_typos=50)  
        for typo in typos:
            f.write(f"{word}\t{typo}\n")